In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
import sys
print(sys.executable)

d:\SuperEngineer\Projects\Github\rag-personal-assistant\venv\Scripts\python.exe


In [4]:
from rag_assistant.utils import get_project_root
import os

project_root = get_project_root()
os.chdir(project_root)
print(f"当前工作目录设置为项目根目录: {project_root}")

当前工作目录设置为项目根目录: D:\SuperEngineer\Projects\Github\rag-personal-assistant


In [5]:
# 加载 .env 文件中的所有变量到环境变量中
load_dotenv()
# 通过 os.getenv 读取变量
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
DATA_DIR = Path("data")
VECTORDB_DIR = Path("vectordb")

In [7]:
def load_documents(data_dir):
    documents = []
    for path in data_dir.rglob("*.pdf"):
        print(f"🔍 Loading: {path}")
        loader = PyPDFLoader(str(path))
        docs = loader.load()
        # 添加文档来源元数据
        for doc in docs:
            doc.metadata["source_path"] = str(path)
            doc.metadata["source_type"] = "pdf"
        documents.extend(docs)
    return documents

In [8]:
def chunk_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", "!", "?", " ", ""]
    )
    return splitter.split_documents(documents)

In [9]:
def embed_documents(chunks):
    embeddings = OpenAIEmbeddings()
    vectordb = FAISS.from_documents(chunks, embeddings)
    vectordb.save_local(str(VECTORDB_DIR))
    print(f"✅ Vector DB saved to: {VECTORDB_DIR}")
    return vectordb

In [10]:
print("📂 Loading documents...")
docs = load_documents(DATA_DIR)

📂 Loading documents...
🔍 Loading: data\pdf\embedded\Bailey - 2005 - Embedded systems desktop integration.pdf
🔍 Loading: data\pdf\embedded\Di Paolo Emilio - 2015 - Embedded Systems Design for High-Speed Data Acquis.pdf
🔍 Loading: data\pdf\embedded\ecoder_ug.pdf
🔍 Loading: data\pdf\embedded\Infineon-AURIX_TC3xx_Part1-UserManual-v01_00-EN.pdf
